In [1]:
import pandas as pd
import numpy as np
import sklearn
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

2024-11-29 22:36:26.945470: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
# In case of errors, check if you are using the correct versions of the libraries. This notebook works on the following versions of libraries. Specify these versions if you get version related errors.
# TensorFlow version: 2.17.0
# Pandas version: 2.1.4
# NumPy version: 1.26.4
# Scikit-learn version: 1.5.2
# Keras version: 3.4.1

# print("TensorFlow version:", tf.__version__)
# print("Pandas version:", pd.__version__)
# print("NumPy version:", np.__version__)
# print("Scikit-learn version:", sklearn.__version__)
# print("Keras version:", tf.keras.__version__)

## Part 1: Import the Housing data and do feature transformations

In [2]:
df= pd.read_csv('house_price_full+(1).csv')
df.head()

,bedrooms,sqft_living,price
0,3,1340,313000
1,5,3650,2384000
2,3,1930,342000
3,3,2000,420000
4,4,1940,550000


In [3]:
X = df.copy()
# Remove target
Y = X.pop('price')

# perform a scaler transform of the input data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# perform log transformation of target variable (For Sandeep: Is this needed?)
Y = np.log(Y)

In [4]:
df_scaled = pd.DataFrame(X)
df_scaled

,0,1
0,-0.433198,-0.753258
1,1.675735,1.457330
2,-0.433198,-0.188649
3,-0.433198,-0.121661
4,0.621269,-0.179079
...,...,...
494,0.621269,0.873582
495,1.675735,2.299459
496,-0.433198,-0.724549
497,-0.433198,-0.179079


In [4]:
Y

0      12.653958
1      14.684290
2      12.742566
3      12.948010
4      13.217674
         ...    
494    13.380102
495    13.764217
496    12.128111
497    12.721886
498    12.254863
Name: price, Length: 499, dtype: float64

## Part 2: Create Model Using `keras`

![](multiple_neurons.png)

In [5]:
from tensorflow import keras

In [6]:
# model = keras.Sequential(
#     [
#         keras.layers.Dense(
#             2, activation="sigmoid", input_shape=(X.shape[-1],)
#         ),
#         keras.layers.Dense(1, activation="linear")
#     ]
# )
# model.summary()

# Newer versions of Keras support this:
model = keras.Sequential()
model.add(keras.layers.Input(shape=(X.shape[-1],)))
model.add(keras.layers.Dense(2, activation="sigmoid"))
model.add(keras.layers.Dense(1, activation="linear"))
model.summary()

2024-11-29 22:37:50.447299: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-29 22:37:50.447518: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 6         
                                                                 
 dense_1 (Dense)             (None, 1)                 3         
                                                                 
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Define the model 
model = keras.Sequential() 
model.add(keras.layers.Dense(64, input_dim=1000, activation='relu')) 
model.add(keras.layers.Dense(32, activation='relu')) 
model.add(keras.layers.Dense(32, activation='relu')) 
model.add(keras.layers.Dense(5, activation='softmax')) 
# Print the model summary model.summary()
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 64)                64064     
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 32)                1056      
                                                                 
 dense_5 (Dense)             (None, 5)                 165       
                                                                 
Total params: 67,365
Trainable params: 67,365
Non-trainable params: 0
_________________________________________________________________


```python
def random_init_params():
    w1 = tf.Variable(tf.random.uniform((2, 2)))
    b1 = tf.Variable(tf.random.uniform((1, 2)))
    w2 = tf.Variable(tf.random.uniform((2, 1)))
    b2 = tf.Variable(tf.random.uniform((1, 1)))
    return w1,b1,w2,b2


def forward_prop(x, w1, b1, w2, b2):
    z1 = tf.matmul(x,w1) + b1
    h1 = tf.math.sigmoid(z1)
    z2 = tf.matmul(h1,w2) + b2
    h2 = z2
    return h2
```

In [8]:
model.compile(
    optimizer=keras.optimizers.SGD(), loss="mean_squared_error"
)

```python
def train(x, y, w1, b1, w2, b2):
    y_true = y
    with tf.GradientTape() as g:
        y_pred = forward_prop(x, w1, b1, w2, b2)

        # loss
        loss = 0.5*(y_true - y_pred)** 2
    
    #Gradient calculation  
    print("**************************************************")
    print("GRADIENTS")
    print("**************************************************")
    gw1, gb1, gw2, gb2 = g.gradient(loss, [w1, b1, w2, b2])
    print(" the gradient for 1st layer weights are:\n",gw1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer weights are:\n",gw2.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 1st layer bias are:\n",gb1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer bias are:\n",gb2.numpy())
    print("--------------------------------------------------")

    # Gradient descent:
    lr=0.2
    w1.assign_sub(lr*gw1)
    b1.assign_sub(lr*gb1)
    w2.assign_sub(lr*gw2)
    b2.assign_sub(lr*gb2)
    print("**************************************************")
    print("NEW UPDATES")
    print("**************************************************")
    print(" the updated 1st layer weights are:\n",w1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer weights are:\n",w2.numpy())
    print("--------------------------------------------------")
    print(" the updated 1st layer bias are:\n",b1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer bias are:\n",b2.numpy())


    return w1, b1, w2, b2,loss

```

In [9]:
model.fit(X,Y.values,epochs=10,batch_size=32)

Epoch 1/10


2024-11-29 22:39:19.578429: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-11-29 22:39:19.609292: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignVariableOp.


16/16 [==============================] - 4s 64ms/step - loss: 103.4968
Epoch 2/10
16/16 [==============================] - 0s 5ms/step - loss: 26.9418
Epoch 3/10
16/16 [==============================] - 0s 5ms/step - loss: 5.4782
Epoch 4/10
16/16 [==============================] - 0s 5ms/step - loss: 1.1312
Epoch 5/10
16/16 [==============================] - 0s 5ms/step - loss: 0.3543
Epoch 6/10
16/16 [==============================] - 0s 5ms/step - loss: 0.2174
Epoch 7/10
16/16 [==============================] - 0s 5ms/step - loss: 0.1884
Epoch 8/10
16/16 [==============================] - 0s 5ms/step - loss: 0.1793
Epoch 9/10
16/16 [==============================] - 0s 5ms/step - loss: 0.1740
Epoch 10/10
16/16 [==============================] - 0s 4ms/step - loss: 0.1702


In [10]:
model.predict(X)[:,0]

2024-11-29 22:39:34.563724: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


16/16 [==============================] - 1s 21ms/step


array([12.918468 , 13.461916 , 13.017931 , 13.028678 , 13.128298 ,
       12.687744 , 12.763641 , 13.2565975, 13.090245 , 13.045553 ,
       12.982748 , 13.286918 , 13.076585 , 12.871147 , 13.349429 ,
       12.89237  , 12.959202 , 13.312798 , 12.923931 , 12.888559 ,
       13.063957 , 13.030798 , 12.989306 , 13.259565 , 13.062072 ,
       13.042187 , 13.080724 , 13.174821 , 13.333992 , 13.112602 ,
       13.053858 , 13.278443 , 13.105642 , 13.272711 , 13.376705 ,
       13.187821 , 12.673825 , 13.005387 , 13.022562 , 13.084827 ,
       12.960915 , 13.037156 , 12.68256  , 12.9925585, 12.742046 ,
       12.8847275, 13.024097 , 13.017931 , 12.905546 , 13.338532 ,
       13.2565975, 13.150721 , 13.12651  , 12.997403 , 13.218069 ,
       13.124575 , 13.173095 , 13.463272 , 12.914802 , 12.804419 ,
       13.338532 , 13.122759 , 13.259565 , 13.104834 , 13.286918 ,
       12.691177 , 12.972769 , 12.855304 , 12.932383 , 13.058243 ,
       13.229099 , 13.1435995, 13.188456 , 13.01326  , 12.9166